In [6]:
from autograd import grad
import autograd.numpy as np
import matplotlib.pyplot as plt
from lista import LISTA, make_loss
from functions import *
from scipy.linalg import hadamard

In [7]:
n_samples = 10000
n_test = 100000
n = 5
p = 10
rng = np.random.RandomState(0)

fit_loss = 'l2'
reg = 'l1'

fit_function, der_function = {
                              'l2': (l2_fit, l2_der),
                              'logreg': (logreg_fit, logreg_der)
                              }[fit_loss]
reg_function, prox = {
                      'l2': (l2_pen, l2_prox),
                      'l1': (l1_pen, l1_prox),
                      None: (no_pen, no_prox)
                      }[reg]
loss = make_loss(fit_function, reg_function)



K = np.eye(p)

z_true = rng.randn(p, n_samples)
D = rng.randn(n, p).dot(K)
D /= np.linalg.norm(D, axis=0, keepdims=True)

sigma = 0.1
eps = rng.randn(n, n_samples)

z_test = rng.randn(p, n_test)
eps_test = rng.randn(n, n_test)
X = np.dot(D, z_true) + eps
X_test = np.dot(D, z_test) + eps_test
if fit_loss == 'logreg':
    X = 2 * (X > 0) - 1
    X_test = 2 * (X_test > 0) - 1

In [9]:
lbda_max = np.max(np.abs(D.T.dot(X)))
lbda = .6 * lbda_max
print(lbda_max)

10.261304163592998


In [10]:
f_min = LISTA(D, lbda, 1000, fit_loss, reg).compute_loss(X_test)

In [11]:
n_layers = 3
lista = LISTA(D, lbda, n_layers, fit_loss, reg, variables='both', learn_levels=False)
z = lista.transform(X_test)
print('frac: %.2e' % (np.sum(np.sum(z != 0 , axis=0) > 0) / X_test.size))
print('loss: %.2e' % (lista.compute_loss(X_test) - f_min))
lista.fit(X, l_rate=1e-1, max_iter=200, verbose=100, backtrack=True)
z = lista.transform(X_test)
print('frac: %.2e' % (np.sum(np.sum(z != 0 , axis=0) > 0) / X_test.size))
print('loss: %.2e' % (lista.compute_loss(X_test) - f_min))

frac: 2.88e-03
loss: 1.37e-03
it 0, loss = 7.449e+00, grad_W = 3.46e-02, grad_l = 4.04e-02, l_rate = 1.00e-01
it 100, loss = 7.448e+00, grad_W = 2.35e-03, grad_l = 5.81e-03, l_rate = 8.00e-01
frac: 4.08e-03
loss: 1.07e-03
